<a href="https://colab.research.google.com/github/Robert0912/Solucion_Sudokus_con_ANN/blob/main/Sudokus_training100k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importación de librerías

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as kr
from keras.optimizers import Adam 


### Funciones

In [17]:
def sumafc(vec):
  constante=9
  sumafil=np.zeros(constante)
  sumacol=np.zeros(constante)
  sumacuad=np.zeros(constante)
  sumas=np.zeros(constante*3)
  for i in range (constante):
    sumafil[i]=sum(vec[0,i:81:constante])
    sumacol[i]=sum(vec[0,constante*i:(constante*i+constante)])
  #print(sumafil)
  #print(sumacol)
  i=0
  #sumacuad[0]=sum(vec[0,constante*i:(constante*i+2):1])+sum(vec[0,constante*(i+1):(constante*i+2):1])
  sumacuad[0]=sum(vec[0,0:3])+sum(vec[0,9:12])+sum(vec[0,18:21]);sumacuad[1]=sum(vec[0,3:6])+sum(vec[0,12:15])+sum(vec[0,21:24]);sumacuad[2]=sum(vec[0,6:9])+sum(vec[0,15:18])+sum(vec[0,24:27])
  sumacuad[3]=sum(vec[0,27:30])+sum(vec[0,36:39])+sum(vec[0,45:48]);sumacuad[4]=sum(vec[0,30:33])+sum(vec[0,39:42])+sum(vec[0,48:51]);sumacuad[5]=sum(vec[0,33:36])+sum(vec[0,42:45])+sum(vec[0,51:54])
  sumacuad[6]=sum(vec[0,54:57])+sum(vec[0,63:66])+sum(vec[0,72:75]);sumacuad[7]=sum(vec[0,57:60])+sum(vec[0,66:69])+sum(vec[0,75:78]);sumacuad[8]=sum(vec[0,60:63])+sum(vec[0,69:72])+sum(vec[0,78:81])
  #print(sumacuad)
  sumas[0:9]=sumafil
  sumas[9:18]=sumacol
  sumas[18:27]=sumacuad
  #print(sumas)
  return sumas

def SeparadorBinario(V,N):
  Z=np.arange(9*N)
  V_Bin=np.zeros_like(Z)
  k=0
  for i in range(N):
    for j in range(9):
      if V[i]==(j+1):
        V_Bin[k]=int(1)
      else:
        V_Bin[k]=int(0)
      k=k+1
  k=0
  return V_Bin

#elecciones=np.zeros((1,729))
def SeleccionNumero(Mat,Nel):
  i=0
  Nsalidas=81
  selecciones=np.zeros((1,Nsalidas))
  for i in range (Nsalidas):
    j=0
    if i==0:
      selecciones[0,i]=np.argmax(Mat[0,0:9])+1
    else:
      selecciones[0,i]=np.argmax(Mat[0,9*i:(9*i+9)])+1
  return selecciones

def EleccionNumero(Mat,Nel):
  i=0
  elecciones=np.zeros((1,729))
  Nsalidas=81
  for i in range (Nsalidas):
    j=0
    if i==0:
      elecciones[0,i+np.argmax(Mat[0,9*i:(9*i+9)])]=1
    else:
      elecciones[0,i*9+np.argmax(Mat[0,9*i:(9*i+9)])]=1
  return elecciones

def accuracy(predicted_labels, actual_labels):
  diff = predicted_labels[0] - actual_labels[0]
  return (float(np.count_nonzero(diff==0)) / len(diff))

def rendimiento(x,y,y_predict):
  diff = y_predict [[0]] - y[[0]]
  iguales_y_vs_ypredict=(float(np.count_nonzero(diff==0)))
  correctos_previos=np.count_nonzero(x[[0]])
  diferencia=iguales_y_vs_ypredict-correctos_previos
  if diferencia>0:
    return diferencia/((81)-correctos_previos)
  else:
    return diferencia/(correctos_previos)

'''def rendimiento(x,y,y_predict):
  diff = y_predict - y
  iguales_y_vs_ypredict=(float(np.count_nonzero(diff==0)))
  correctos_previos=np.count_nonzero(x)
  diferencia=iguales_y_vs_ypredict-correctos_previos
  if diferencia>0:
    return diferencia/(len(x)-correctos_previos)
  else:
    return diferencia/(correctos_previos)'''



'def rendimiento(x,y,y_predict):\n  diff = y_predict - y\n  iguales_y_vs_ypredict=(float(np.count_nonzero(diff==0)))\n  correctos_previos=np.count_nonzero(x)\n  diferencia=iguales_y_vs_ypredict-correctos_previos\n  if diferencia>0:\n    return diferencia/(len(x)-correctos_previos)\n  else:\n    return diferencia/(correctos_previos)'

### Lectura del dataset

In [5]:
#Cargar los datos .csv
numtest=25000
numdatos=100000+numtest
#df=pd.read_csv(path + 'sudoku.csv', sep=',',header=0)
sudokus = next(pd.read_csv('sudoku.csv', chunksize=(numdatos))).values
print(sudokus)


[['004300209005009001070060043006002087190007400050083000600000105003508690042910300'
  '864371259325849761971265843436192587198657432257483916689734125713528694542916378']
 ['040100050107003960520008000000000017000906800803050620090060543600080700250097100'
  '346179258187523964529648371965832417472916835813754629798261543631485792254397186']
 ['600120384008459072000006005000264030070080006940003000310000050089700000502000190'
  '695127384138459672724836915851264739273981546946573821317692458489715263562348197']
 ...
 ['005700009090003400800016205400050600007090302206801070060080001700402503540000000'
  '325748169691523487874916235439257618187694352256831974962385741718462593543179826']
 ['900004020020710038108500906000250013870006000050400809000600105600020080390007040'
  '937864521526719438148532976469258713871396254253471869782643195614925387395187642']
 ['180600400040005309000097820900000057650023008001000200410056090037014000500208003'
  '18963247574218536936549782192384165765472

In [6]:
quizzes, solutions = sudokus.T
#flatX = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9)) for flatten_grid in quizzes])
a =  np.array([np.reshape([int(d) for d in flatten_grid], (1, 81)) for flatten_grid in quizzes])
b =  np.array([np.reshape([int(d) for d in flatten_grid], (1, 81)) for flatten_grid in solutions])
aux_a=a[0];
aux_b=b[0];
for i in range(len(a)-1):
    aux_a=np.vstack((aux_a,a[i+1]))
    aux_b=np.vstack((aux_b,b[i+1]))
sudokus_inicio=aux_a
sudokus_solucion=aux_b

print(sudokus_inicio.shape)
print(sudokus_solucion.shape)

(125000, 81)
(125000, 81)


In [8]:
numentradas=729
x1=np.zeros((numdatos,numentradas))
#y=np.zeros((numdatos,numentradas))
y1=np.zeros((numdatos,729))
for i in range(int(sudokus_inicio.size/81)):
  x1[i,0:729]=SeparadorBinario(np.transpose(sudokus_inicio[[i]]),sudokus_inicio[[i]].size)
  #x[i,729:756]=sumafc(sudokus_inicio[[i]])
  y1[i,0:729]=SeparadorBinario(np.transpose(sudokus_solucion[[i]]),sudokus_solucion[[i]].size)
  #y[i,729:756]=sumafc(sudokus_solucion[[i]])
#print(x[[0]]==x[[100]])
#print(x[[0]]==SeparadorBinario(np.transpose(sudokus_inicio[[0]]),sudokus_inicio[[0]].size))
#print(y[[0]])
#print(np.reshape(sudokus_inicio[[0]],(9,9)))

x=x1[0:numdatos-numtest,:]
y=y1[0:numdatos-numtest,:]

xtest=x1[numdatos-numtest:numdatos,:]
ytest=y1[numdatos-numtest:numdatos,:]
print(ytest.shape)

(25000, 729)


In [ ]:
#from IPython.core.display import display, HTML
# learning rate
#nn = [81, 200, 200, 729, 81]  # número de neuronas por capa. note que acá están las de entrada y salida, no solo las ocultas
#nn = [81, 200, 200, 81]
#nn = [numentradas, 2000, 2000, 1800, 729]
nn = [numentradas, 2000, 2000, 729]

# Creamos el objeto que contendrá a nuestra red neuronal, como
# secuencia de capas.
model = kr.Sequential()
adam = Adam(lr=.001)
# Añadimos la capa 1
l1 = model.add(kr.layers.Dense(nn[1], activation='relu'))

# Añadimos la capa 2
l2 = model.add(kr.layers.Dense(nn[2], activation='relu'))

# Añadimos la capa 3
l3 = model.add(kr.layers.Dense(nn[3], activation='softmax'))

# Añadimos la capa 4
#l4 = model.add(kr.layers.Dense(nn[4]))

# Compilamos el modelo, definiendo la función de coste y el optimizador.
#model.compile(loss='mape', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['binary_crossentropy'])
model.compile(loss='binary_crossentropy', optimizer=adam)
#model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

#model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_crossentropy'])
#probar loss 'mae'




# Y entrenamos al modelo. Los callbacks 
#model.fit(X_train, y_train, epochs=1000,batch_size=10)
model.fit(x, y, epochs=500,batch_size=10000)
#model.fit(X_train, y_train, epochs=1000)

Epoch 1/500
10/10 [==============================] - 59s 6s/step - loss: 0.7313
Epoch 2/500
10/10 [==============================] - 59s 6s/step - loss: 0.7139
Epoch 3/500
10/10 [==============================] - 59s 6s/step - loss: 0.6837
Epoch 4/500
10/10 [==============================] - 59s 6s/step - loss: 0.6566
Epoch 5/500
10/10 [==============================] - 59s 6s/step - loss: 0.6420
Epoch 6/500
10/10 [==============================] - 59s 6s/step - loss: 0.6350
Epoch 7/500
10/10 [==============================] - 59s 6s/step - loss: 0.6286
Epoch 8/500
10/10 [==============================] - 59s 6s/step - loss: 0.6244
Epoch 9/500
10/10 [==============================] - 59s 6s/step - loss: 0.6210
Epoch 10/500
10/10 [==============================] - 59s 6s/step - loss: 0.6182
Epoch 11/500
10/10 [==============================] - 59s 6s/step - loss: 0.6162
Epoch 12/500
10/10 [==============================] - 59s 6s/step - loss: 0.6137
Epoch 13/500
10/10 [=================

In [9]:
model=kr.models.load_model("modelo_training100k.h5") 

In [10]:
yyyy=np.zeros((1,numentradas))
yyy=np.array([[0,0,4,3,0,0,2,0,9,0,0,5,0,0,9,0,0,1,0,7,0,0,6,0,0,4,3,0,0,6,0,0,2,0,8,7,1,9,0,0,0,7,4,0,0,0,5,0,0,8,3,0,0,0,6,0,0,0,0,0,1,0,5,0,0,3,5,0,8,6,9,0,0,4,2,9,1,0,3,0,0]])
#yyy=np.array([[4,5,0,0,0,0,0,0,0,0,0,1,6,0,0,7,9,0,3,0,7,2,0,0,1,0,0,0,0,0,8,5,0,9,0,2,0,0,2,1,6,9,3,0,4,5,0,0,0,0,0,0,8,0,0,6,8,0,9,4,0,0,1,0,7,0,5,0,8,0,6,0,0,0,0,0,1,0,0,2,3]])
yy=np.transpose(yyy)
print(np.reshape(yyy,(9,9)))

#x[[i]]=SeparadorBinario(np.transpose(sudokus_inicio[[i]]),sudokus_inicio[[i]].size)

#prediccion=model.predict(yyy)
#prediccion_proba=model.predict_proba(yyy)
yyyy[0,0:729]=SeparadorBinario(np.transpose(yyy),yyy.size)
#yyyy[0,729:756]=sumafc(yyy)
print(yyyy.shape)

prediccion=model.predict(yyyy)
prediccion_proba=model.predict_proba(yyyy)

#pred=np.round(prediccion)
#print(np.reshape(pred,(9,9))

#print(np.reshape(prediccion,(9,9)))
#print(np.reshape(prediccion_proba,(27,27)))


print(prediccion_proba[0,0:9])
#print(prediccion_proba.shape)
print(prediccion[0,0:9])


[[0 0 4 3 0 0 2 0 9]
 [0 0 5 0 0 9 0 0 1]
 [0 7 0 0 6 0 0 4 3]
 [0 0 6 0 0 2 0 8 7]
 [1 9 0 0 0 7 4 0 0]
 [0 5 0 0 8 3 0 0 0]
 [6 0 0 0 0 0 1 0 5]
 [0 0 3 5 0 8 6 9 0]
 [0 4 2 9 1 0 3 0 0]]
(1, 729)
Instructions for updating:
Please use `model.predict()` instead.
[0.         0.0006911  0.         0.         0.00014534 0.
 0.00016936 0.         0.        ]
[0.         0.0006911  0.         0.         0.00014534 0.
 0.00016936 0.         0.        ]


In [11]:
respuesta=np.reshape(SeleccionNumero(prediccion,prediccion.size),(9,9))
print(respuesta)

[[2. 9. 2. 3. 6. 7. 1. 1. 5.]
 [1. 1. 1. 1. 4. 1. 6. 7. 4.]
 [6. 1. 5. 6. 1. 4. 5. 1. 7.]
 [1. 2. 5. 1. 4. 1. 5. 2. 2.]
 [1. 5. 2. 1. 5. 9. 4. 6. 3.]
 [2. 5. 5. 8. 9. 6. 9. 6. 1.]
 [1. 3. 1. 5. 1. 1. 1. 1. 5.]
 [7. 7. 3. 4. 8. 9. 8. 7. 7.]
 [1. 1. 4. 9. 4. 5. 3. 1. 1.]]


In [ ]:
'''y_true = [[0., 1.], [0., 0.]]
y_pred = [[0.0006, 0.0004], [0.000067, 0.00043]]
# Using 'auto'/'sum_over_batch_size' reduction type.
bce = tf.keras.losses.BinaryCrossentropy()
bce(y_true, y_pred).numpy()'''

"y_true = [[0., 1.], [0., 0.]]\ny_pred = [[0.0006, 0.0004], [0.000067, 0.00043]]\n# Using 'auto'/'sum_over_batch_size' reduction type.\nbce = tf.keras.losses.BinaryCrossentropy()\nbce(y_true, y_pred).numpy()"

In [12]:
yyyy1=np.zeros((1,numentradas))
yyy1=np.array([[0,4,0,1,0,0,0,5,0,1,0,7,0,0,3,9,6,0,5,2,0,0,0,8,0,0,0,0,0,0,0,0,0,0,1,7,0,0,0,9,0,6,8,0,0,8,0,3,0,5,0,6,2,0,0,9,0,0,6,0,5,4,3,6,0,0,0,8,0,7,0,0,2,5,0,0,9,7,1,0,0]])

print(yyy1.shape)

yyyy1[0,0:729]=SeparadorBinario(np.transpose(yyy1),yyy1.size)
#yyyy1[0,729:756]=sumafc(yyyy1)
prediccion1=model.predict(yyyy1)

respuesta1=np.reshape(SeleccionNumero(prediccion1,prediccion1.size),(9,9))

print(respuesta1)
#346179258187523964529648371965832417472916835813754629798261543631485792254397186

(1, 81)
[[7. 9. 2. 1. 2. 7. 1. 1. 4.]
 [1. 1. 1. 1. 4. 1. 6. 8. 4.]
 [6. 1. 5. 6. 1. 4. 4. 1. 7.]
 [1. 2. 5. 8. 4. 1. 5. 2. 1.]
 [1. 7. 2. 1. 1. 9. 4. 4. 5.]
 [2. 5. 5. 8. 5. 5. 9. 1. 1.]
 [1. 3. 1. 5. 1. 1. 1. 1. 8.]
 [7. 7. 4. 4. 8. 9. 8. 7. 9.]
 [1. 1. 4. 9. 4. 5. 3. 1. 1.]]


In [13]:
yyyy2=np.zeros((1,numentradas))
yyy2=np.array([[0,7,0,0,0,0,4,0,9,4,1,0,0,5,0,0,0,0,0,0,5,3,0,0,0,2,7,3,0,4,8,0,0,5,0,0,0,2,8,0,3,6,0,9,0,7,0,6,4,0,9,0,1,0,0,0,0,9,0,0,0,0,0,6,0,2,1,8,0,0,3,0,0,0,0,0,4,5,8,7,6]])
print(yyy2.shape)

yyyy2[0,0:729]=SeparadorBinario(np.transpose(yyy2),yyy2.size)
#yyyy2[0,729:756]=sumafc(yyyy2)
prediccion2=model.predict(yyyy2)

respuesta2=np.reshape(SeleccionNumero(prediccion2,prediccion2.size),(9,9))

print(respuesta2)

#070000409 - 273618459
#410050000 - 419752683
#005300027 - 865394127
#304800500 - 394871562
#028036090 - 128536794
#706409010 - 756429318
#000900000 - 587963241
#602180030 - 642187935
#000045876 - 931245876




(1, 81)
[[2. 9. 2. 3. 6. 7. 1. 1. 5.]
 [1. 1. 1. 1. 1. 1. 6. 8. 4.]
 [6. 1. 5. 6. 1. 4. 4. 1. 7.]
 [1. 2. 5. 8. 4. 1. 5. 2. 2.]
 [1. 5. 5. 1. 1. 9. 4. 4. 7.]
 [2. 5. 5. 8. 9. 5. 9. 1. 8.]
 [1. 3. 1. 5. 1. 1. 1. 1. 5.]
 [7. 7. 4. 3. 8. 9. 8. 7. 9.]
 [1. 1. 4. 9. 4. 5. 9. 1. 1.]]


In [18]:
print(xtest.shape)
#predictions=np.zeros((1,numdatos))
acc=np.zeros(numtest)
ren=np.zeros(numtest)
#prediction=SeleccionNumero(model.predict(xtest[[0]]),model.predict(xtest[[0]]).size)
for i in range (numtest):
  #elecciones=np.zeros((1,729))
  prediction=SeleccionNumero(model.predict(xtest[[i]]),model.predict(xtest[[i]]).size)
  acc[i]=accuracy(prediction,sudokus_solucion[[numdatos-numtest+i]])
  ren[i]=rendimiento(sudokus_inicio[[numdatos-numtest+i]],sudokus_solucion[[numdatos-numtest+i]],prediction)
  #print(acc)

print("Accuracy: ",acc)
print("Rendimiento: ",acc)

#print(np.reshape(sudokus_inicio[[1000]],(9,9)))
#print(np.reshape(prediction,(9,9)))
print(elecciones.shape)
print(ytest[[-1]].shape)

(25000, 729)
Accuracy:  [0.08641975 0.13580247 0.08641975 ... 0.17283951 0.16049383 0.12345679]
Rendimiento:  [0.08641975 0.13580247 0.08641975 ... 0.17283951 0.16049383 0.12345679]
(1, 729)
(1, 729)


In [19]:
print("Accuracy: ",acc)
accmax=max(acc)
accmin=min(acc)
accmean=sum(acc)/numtest
print("La exactitud máxima es: ",accmax,"\nLa exactitud mínima es: ",accmin,"\nLa exactitud del conjunto de validación es: ", accmean)

Accuracy:  [0.08641975 0.13580247 0.08641975 ... 0.17283951 0.16049383 0.12345679]
La exactitud máxima es:  0.32098765432098764 
La exactitud mínima es:  0.037037037037037035 
La exactitud del conjunto de validación es:  0.16636888888887885


In [20]:
print("Rendimiento: ",ren)
renmax=max(ren)
renmin=min(ren)
renmean=sum(ren)/numtest
print("El rendimiento máximo es: ",renmax,"\nEl rendimiento mínimo es: ",renmin,"\nEl rendimiento del conjunto de validación es: ", renmean)

Rendimiento:  [-0.8        -0.66666667 -0.79411765 ... -0.57575758 -0.61764706
 -0.71428571]
El rendimiento máximo es:  -0.1935483870967742 
El rendimiento mínimo es:  -0.9142857142857143 
El rendimiento del conjunto de validación es:  -0.6011834649984351


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2000)              1460000   
_________________________________________________________________
dense_1 (Dense)              (None, 2000)              4002000   
_________________________________________________________________
dense_2 (Dense)              (None, 729)               1458729   
Total params: 6,920,729
Trainable params: 6,920,729
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Guardar el Modelo
model.save('modelo_training100k.h5')

# Recrea exactamente el mismo modelo solo desde el archivo
#new_model = keras.models.load_model('path_to_my_model.h5')
